In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Loading Dataset
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#checking counts-Rows&columns
dataset.shape

(400, 5)

In [4]:
#get dummies
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
#dropping the column that is not required which has no information
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
#How many classes we are going to classify
dataset["Purchased"].value_counts()


Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
#spliting Independent and Dependent
Independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
Dependent=dataset["Purchased"]

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Independent,Dependent, test_size = 1/3, random_state = 0)


In [9]:
#StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [10]:
#model creation
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['rbf','poly','sigmoid'],'gamma':[None,'scale'],'C':[10,50]} 
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
        

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
23 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_base.py", line 180, in fit
    self._validate_params()
  File "C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 50], 'gamma': [None, 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [12]:
grid_predictions = grid.predict(X_test)
        

In [13]:
#evaluation metrics(confusion_matrix)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
        

In [14]:
print(cm)

[[82  3]
 [24 25]]


In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86        85
           1       0.89      0.51      0.65        49

    accuracy                           0.80       134
   macro avg       0.83      0.74      0.75       134
weighted avg       0.82      0.80      0.78       134



In [17]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001296,0.001089,0.000000,0.000000,10,None,rbf,"{'C': 10, 'gamma': None, 'kernel': 'rbf'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,0.000386,0.000557,0.000000,0.000000,10,None,poly,"{'C': 10, 'gamma': None, 'kernel': 'poly'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.001593,0.001200,0.000000,0.000000,10,None,sigmoid,"{'C': 10, 'gamma': None, 'kernel': 'sigmoid'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
3,0.018640,0.005008,0.002963,0.003721,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.782557,0.744194,0.651089,0.805463,0.747667,0.746194,0.052700,2
4,0.030920,0.001673,0.003767,0.002380,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.776290,0.717410,0.621589,0.778706,0.721829,0.723165,0.057026,3
5,0.016975,0.003884,0.007061,0.002741,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.686940,0.664320,0.682050,0.775815,0.480346,0.657894,0.096847,5
6,0.001054,0.000709,0.000000,0.000000,50,None,rbf,"{'C': 50, 'gamma': None, 'kernel': 'rbf'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,0.001121,0.000361,0.000000,0.000000,50,None,poly,"{'C': 50, 'gamma': None, 'kernel': 'poly'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
8,0.001639,0.000748,0.000000,0.000000,50,None,sigmoid,"{'C': 50, 'gamma': None, 'kernel': 'sigmoid'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.021179,0.005836,0.005600,0.003971,50,scale,rbf,"{'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}",0.782557,0.744194,0.679033,0.805463,0.747667,0.751783,0.042885,1


In [18]:
#f1 score prediction

from sklearn.metrics import f1_score
f1_weighted=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_weighted value for best parameter {}:".format(grid.best_params_),f1_weighted)

The f1_weighted value for best parameter {'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}: 0.7821080224768587


In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8168067226890756